In [1]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.enums import *

from quantfreedom.simulate import run_df_backtest, or_backtest
from quantfreedom.strategies.strategy import Strategy


mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [2]:
candles_np = mufex_main.get_candles(symbol="BTCUSDT", timeframe="5m", candles_to_dl=500)

It took 00 mins and 00 seconds to download 500 candles


In [3]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
# backtest_settings = BacktestSettings(
#     array_size=200,
#     gains_pct_filter=-90,
#     total_trade_filter=10,
#     upside_filter=-np.inf,
# )
backtest_settings = BacktestSettings()
dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([9, 12]),
    max_trades=np.array([5]),
    num_candles=np.array([100]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2, 5]),
    sl_based_on_add_pct=np.array([0.01]),
    sl_based_on_lookback=np.array([20]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    static_leverage=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5]),
    trail_sl_when_pct=np.array([1]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    logger_bool=True,
    long_or_short=LongOrShortType.Long,
    pg_min_max_sl_bcb=PriceGetterType.Min,
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_equity=1000.0,
    tp_fee_type=TakeProfitFeeType.Limit,
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=ZeroOrEntryType.Nothing,
)
strategy = Strategy(
    candle_processing_type=CandleProcessingType.Backtest,
    rsi_is_below=np.array([100]),
    rsi_period=np.array([14]),
)

In [4]:
strat_records_df = run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    static_os=static_os,
    strategy=strategy,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 1
Total order settings to test: 4
Total combinations of settings to test: 4
Total candles: 500
Total candles to test: 2,000


In [5]:
strat_records_df

,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,to_the_upside,fees_paid,total_pnl,ending_eq
0,0,0,5.0,1,4,-15.215,20.000,-0.000,160.782,-152.152,847.848
1,0,1,5.0,1,4,-1.407,20.000,-0.000,160.824,-14.071,985.929
2,0,2,5.0,1,4,-18.915,20.000,-0.000,204.373,-189.155,810.845
3,0,3,6.0,1,5,-11.943,16.667,-0.029,220.427,-119.427,880.573


In [6]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    static_os=static_os,
    strategy=strategy,
    dos_index=2,
    ind_set_index=0
)

In [7]:
order_records_df

,ind_set_idx,or_set_idx,bar_idx,timestamp,datetime,order_status,equity,available_balance,cash_borrowed,cash_used,...,entry_size_usd,entry_price,exit_price,position_size_asset,position_size_usd,realized_pnl,sl_pct,sl_price,tp_pct,tp_price
0,0,2,100,1699087200000,2023-11-04 08:40:00,EntryFilled,1000.000,909.687,7382.438,90.313,...,7472.751,34783.3,NaN,0.215,7472.751,NaN,0.2,34706.2,0.9,35104.4
1,0,2,101,1699087500000,2023-11-04 08:45:00,EntryFilled,1000.000,818.963,14711.683,181.037,...,7510.282,34782.6,NaN,0.431,14983.033,NaN,0.2,34706.2,0.9,35103.5
2,0,2,102,1699087800000,2023-11-04 08:50:00,EntryFilled,1000.000,750.569,19988.468,249.431,...,5526.216,34832.3,NaN,0.590,20509.249,NaN,0.3,34706.2,1.0,35140.7
3,0,2,103,1699088100000,2023-11-04 08:55:00,EntryFilled,1000.000,683.186,24995.943,316.814,...,5324.289,34850.9,NaN,0.743,25833.538,NaN,0.3,34706.2,1.0,35168.5
4,0,2,138,1699098600000,2023-11-04 11:50:00,StopLossFilled,881.374,881.374,NaN,NaN,...,NaN,NaN,34706.2,NaN,NaN,-118.626,NaN,NaN,NaN,NaN
5,0,2,139,1699098900000,2023-11-04 11:55:00,EntryFilled,881.374,765.092,9960.121,116.282,...,10076.403,34699.6,NaN,0.290,10076.403,NaN,0.1,34672.5,0.6,34920.4
6,0,2,139,1699098900000,2023-11-04 11:55:00,StopLossFilled,855.408,855.408,NaN,NaN,...,NaN,NaN,34672.5,NaN,NaN,-25.966,NaN,NaN,NaN,NaN
7,0,2,140,1699099200000,2023-11-04 12:00:00,EntryFilled,855.408,752.649,8801.864,102.759,...,8904.623,34706.1,NaN,0.257,8904.623,NaN,0.1,34667.2,0.7,34950.5
8,0,2,141,1699099500000,2023-11-04 12:05:00,EntryFilled,855.408,643.321,18063.634,212.087,...,9473.857,34700.0,NaN,0.530,18378.480,NaN,0.1,34667.2,0.7,34942.1
9,0,2,142,1699099800000,2023-11-04 12:10:00,EntryFilled,855.408,520.174,28399.686,335.234,...,10671.286,34687.5,NaN,0.838,29049.766,NaN,0.1,34667.2,0.7,34927.6
